In [ ]:
!wget --no-cache -O x-rays.zip.npy -q "https://drive.google.com/uc?export=download&id=1It4NYRJNEem3YLHJqYZyclUbR85eqZH8"

In [ ]:
!pip install opencv-python

In [ ]:
from tqdm import tqdm
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
from glob import glob
import sklearn
import skimage
from skimage.transform import resize
import random
from cv2.gapi import RGB2Gray
from skimage.color import rgb2gray
import pdb

La siguiente función solo opera de forma local lo cual facilita la obtención de las imágenes, a partir de un archivo llamado x-rays.zip y el cual contiene los directorios con las imágenes de rayos x, debidamente clasificados.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Ruta al archivo .zip que deseas descomprimir
archivo_zip = './drive/MyDrive/Diplomado En Machine Learning/Módulo 5 - Redes Neuronales/Proyecto Aplicado/x-rays.zip'

# Directorio de destino donde se descomprimirán los archivos (la misma carpeta)
directorio_destino = '/content/'

# Abre el archivo .zip
with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Extrae todos los archivos en el directorio de destino
    zip_ref.extractall(directorio_destino)

print(f'Archivos descomprimidos en: {directorio_destino}')

Archivos descomprimidos en: /content/


In [ ]:
files = os.listdir("./Data_blance")

# Imprimir el contenido de la carpeta

for file in files:
    print(file)

Test
Val
Train


In [ ]:
train_dir = "Data_blance/Train"
test_dir = "Data_blance/Test"

load_from_images = True

def get_data(folder):
    """
    Carga imágenes desde una carpeta y las prepara para su uso en entrenamiento y prueba.

    Parameters:
    - folder (str): Ruta de la carpeta que contiene subcarpetas con imágenes.

    Returns:
    - X (numpy.ndarray): Matriz de características que representa las imágenes.
    - y (numpy.ndarray): Matriz de etiquetas correspondientes a las clases de las imágenes.

    Note:
    - Las clases se asignan de acuerdo con las subcarpetas 'bacteria', 'Normal', 'virus', y otras.
    - Las imágenes se redimensionan a 150x150 píxeles y se convierten a escala de grises.

    Example:
    ```python
    X_train, y_train = get_data('/ruta/a/los/datos_de_entrenamiento')
    X_test, y_test = get_data('/ruta/a/los/datos_de_prueba')
    ```
    """

    X = []  # Lista para almacenar las imágenes como matrices
    y = []  # Lista para almacenar las etiquetas de clase

    # Iterar a través de las subcarpetas en la carpeta proporcionada
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            # Asignar etiquetas de clase según el nombre de la subcarpeta
            if folderName in ['bacteria']:
                label = 0
            elif folderName in ['Normal']:
                label = 1
            elif folderName in ['virus']:
                label = 2
            else:
                label = 3

            # Iterar a través de los archivos de imágenes en la subcarpeta actual
            for image_filename in tqdm(os.listdir(folder + '/' + folderName)):
                # Leer la imagen utilizando OpenCV
                img_file = cv2.imread(folder + '/' + folderName + '/' + image_filename)
                
                # Verificar si la imagen se cargó correctamente
                if img_file is not None:
                    # Redimensionar la imagen y convertirla a escala de grises
                    img_file = skimage.transform.resize(img_file, (150, 150, 3), mode='constant')
                    img_file = rgb2gray(img_file)
                    
                    # Convertir la imagen a una matriz numpy y agregarla a la lista X
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    
                    # Agregar la etiqueta correspondiente a la lista y
                    y.append(label)

    # Convertir las listas X e y a matrices numpy
    X = np.asarray(X)
    y = np.asarray(y)

    return X, y

# Ejemplo de uso condicional basado en la variable load_from_images
if load_from_images:
    X_train, y_train = get_data(train_dir)
    X_test, y_test = get_data(test_dir)

    # Guardar matrices de entrenamiento y prueba en archivos numpy
    np.save('xtrain.npy', X_train)
    np.save('ytrain.npy', y_train)
    np.save('xtest.npy', X_test)
    np.save('ytest.npy', y_test)
else:
    # Cargar matrices previamente guardadas desde archivos numpy
    X_train = np.load('xtrain.npy')
    y_train = np.load('ytrain.npy')
    X_test = np.load('xtest.npy')
    y_test = np.load('ytest.npy')

100%|██████████| 100/100 [00:17<00:00,  5.87it/s]
